# <b> Canal de diffusion anonyme et génération de secret </b>

<i> Compte-rendu du TP2 par Valentin SÉRÉE et Jérémy SERENA [INFO3]</i>

<h2> <u>Partie 1 :</u> Canal de diffusion anonyme </h2>

In [ ]:
from datetime import datetime

class Forum:
    messages = []

    def init(self):
        self.messages = []

    def recupDate(self): # Récupère la date courante
        date = str(datetime.now().time())
        return date

    def calculDate(self, dateAConvertir): # Convertit une string en date (car les dates sont stockées au format string)
        dateConvertie = datetime.strptime(dateAConvertir, "%H:%M:%S.%f").time()
        return dateConvertie
    
    def recupererMessagesAnonymes(self, date_debut, date_fin) : # Récupère tous les messages situés entre deux dates
        listeMessages = [] # Messages concernés à renvoyer
        for msg in self.messages : # On regarde chaque message posté sur le forum
            date = self.calculDate(msg[0]) # On convertit la date (stockée en string) en une date
            if date > date_debut and date < date_fin : # On regarde si la date du message est comprise entre les deux dates
                listeMessages.append(msg) # Si oui, on l'ajoute aux messages à renvoyer
        return listeMessages

    def posterMessageAnonyme(self, message) : # Ajoute un message au forum anonymement
        date = self.recupDate() # On récupère la date du message
        self.messages.append([date, message]) # On ajoute le message daté
        return date



In [ ]:
# Main à exécuter pour pouvoir interagir avec le forum
# Une barre de saisie va être proposée à l'interlocuteur
# Il pourra alors soit :
    # - écrire un message anonymement
    # - lire les messages postés sur le forum (utiliser "lecture")
        # (il faut pour cela entrer 2 heures au format HH:MM:SS)
    # - arrêter la communication (utiliser "fin_convo")

if __name__ == "__main__" :
    print("Ouverture du forum")
    print("Vous pouvez communiquer sur ce forum (anonymement)")
    print("Pour lire des messages du forum, tapez 'lecture'")
    print("Pour arrêter la discussion, tapez 'fin_convo'")
    forum = Forum()
    discussion=True
    while(discussion) :
        string = input()
        if "lecture" == string :
            print("Entrez la date de début au format HH:MM:SS" )
            date_debut = input()
            print("Entrez la date de fin au format HH:MM:SS" )
            date_fin = input()
            begin = forum.calculDate(date_debut + ".000000")
            end = forum.calculDate(date_fin + ".000000")
            print(forum.recupererMessagesAnonymes(begin, end))
        elif string == "fin_convo":
            discussion=False
        else:
            forum.posterMessageAnonyme(string)

<h2> <u>Partie 2 :</u> Génération de secret via canal de diffusion anonyme </h2>

In [ ]:
import time
import random
import threading

ver = threading.Lock()

class User() :

    def __init__(self) :
        self.historique = {}
        self.name = ""
        self.partner = ""

    # Fonction pour générer un secret :
    # le participant tire un bit et envoie ensuite un des deux noms
    # Se déroule pendant un temps donné
    def genererSecret(self, prenom1, prenom2, temps):
        self.name = prenom1 # le prénom 1 est le prénom de l'utilisateur
        self.partner = prenom2 # le prénom 2 est le prénom de son interlocuteur
        fin = time.time() + temps # on fixe l'heure de fin
        while (time.time() < fin): # tant que l'heure de fin n'est pas atteinte
            attente = 0.1 * random.randint(1, 10) # on définit une attente entre 1 et 10ms
            bit = random.randint(0, 1) # on tire au sort un bit (0 ou 1)
            if bit == 0 : # si le bit est 0, l'utilisateur renverra son prénom
                message = self.name
            else : # sinon, il renvoie le prénom de son adversaire
                message = self.partner
            global ver
            ver.acquire()
            date = self.forum.posterMessageAnonyme(message) # l'utilisateur envoie son message anonymement sur le forum
            self.historique[date] = message # il ajoute ce message à son historique personnel
            time.sleep(attente) # il attend X millisecondes
            ver.release()

    # Fonction pour extraire un secret :
    # le participant accède à la liste des messages envoyés par les 2 interlocuteurs
    # Il compare ses envois aux envois globaux et en déduit un secret commun
    def extraireSecret(self, dateDebut, dateFin):
        secret = "" 
        messages = forum.recupererMessagesAnonymes(dateDebut, dateFin) # L'utilisateur récupère les messages envoyés
        for i in range (len(messages)): # On parcourt tous les messages envoyés
            if messages[i][0] in self.historique: # si le message daté a été envoyé par lui
                if (messages[i][1] == self.name): # si le message comporte son prénom (et qu'il est envoyé par lui)
                    secret+="0" # on ajoute 0 au secret
                else: # si le message comporte le prénom de son partenaire
                    secret+="1" # on ajoute 1 au secret
            else: # si le message est envoyé par le partenaire
                if (messages[i][1] == self.partner): # si le message comporte le prénom de son partenaire
                    secret+="0" # on ajoute 0 au secret
                else: # si le message comporte son prénom
                    secret+="1" # on ajoute 1 au secret
        return secret 

In [ ]:
# =============================================================
# VARIABLE À METTRE À JOUR POUR MODIFIER LA DURÉE DE L'ÉCHANGE

TEMPS_EX = 10 

# =============================================================

# Fonctions pour exécuter les threads en parallèle (et avoir ainsi un vrai échange simultané)
def exec1() :
    global TEMPS_EX
    threading.Thread(bob.genererSecret("Bob", "Alice", TEMPS_EX)).start()

def exec2() :
    global TEMPS_EX
    threading.Thread(alice.genererSecret("Alice", "Bob", TEMPS_EX)).start()

In [ ]:
# Main à exécuter pour lancer la discussion et la génération du secret
# ATTENTION : il faut avant cela exécuter les zones de code suivantes :
    # - class Forum() [1ère zone de code]
    # - class User() [3ème zone de code]
    # - def exec1() et def exec2() [4ème zone de code]

if __name__ == "__main__" :
    forum = Forum() # On instancie la classe Forum
    bob = User() # On instancie deux nouveaux Users
    alice = User()
    bob.forum = forum # On fournit à ces Users le forum de communication
    alice.forum = forum
    dateDebut = datetime.now().time() # On récupère la date de début de l'échange
    time.sleep(2)
    delay1, delay2 = random.randint(0,2), random.randint(0,2) # On règle les paramètres des threads
    t = threading.Timer(delay1, exec1)
    u = threading.Timer(delay2, exec2)
    t.start() # On lance ces 2 threads
    u.start()
    time.sleep(2)
    t.join() # Les threads rejoignent le thread d'exécution principal une fois terminés
    u.join()
    dateFin = datetime.now().time() # On récupère la date de fin de l'échange 
    secretAlice = alice.extraireSecret(dateDebut, dateFin) # Les deux interlocuteurs peuvent alors obtenir les messages envoyés
    secretBob=bob.extraireSecret(dateDebut, dateFin)
    print("Le secret d'Alice est " + secretAlice) # On imprime les secrets (UNIQUEMENT DANS LE CADRE DE L'EXERCICE)
    print("Le secret de Bob est  " + secretBob)

<h2> <u>Partie 3 :</u> Analyse du protocole </h2>

<b> <u> Question 1 </u> </b>

Un tel canal de diffusion anonyme peut être utilisé comme une base pour un système de vote en ligne. On peut donner au forum un ensemble de jetons tous différents et les répartir aléatoirement aux différents participants. Chaque participant envoyant un message doit y joindre un jeton, qui est lu par le forum. 
Si le jeton est valide, le message est comptabilisé. Si le jeton a déjà été utilisé ou n'existe pas, le message est refusé. On garantit donc l'unicité et l'anonymat des votes.

On peut également utiliser ce type de canal pour récolter des avis anonymes sur un produit. Le vendeur du produit ajoute les nouveaux acheteurs dans le groupe, puis affiche régulièrement les nouveaux messages sur sa vitrine. 
Il évite ainsi les avis de ceux n'ayant pas acheté le produit, et les acheteurs peuvent donner un avis de manière anonyme si sufisamment de produits ont été achetés, et une date de publication aléatoire empêche le vendeur de lier un achat récent à une critique.


<b> <u> Question 2 </u> </b>

Bob et Alice obtiennent le même secret. En effet, chacun peut reconnaître les messages qu'il a envoyé (il sait donc à quoi correspondent ces messages dans le secret entre un 0 et un 1). 
Ensuite, à partir des messages observés qu'il n'a pas envoyés, l'utilisateur choisit un 1 s'il voit son nom et un 0 sinon. Les utilisateurs ont donc un même secret puisque la démarche est symétrique de part et d'autre.

L'adversaire n'a la vision que sur l'historique des messages, et a donc, pour chaque bit, une chance sur deux de déterminer avec justesse l'envoyeur du message et donc la valeur du bit. La probabilité de trouver le bon mot de passe en regardant uniquement l'historique est de (1/2^N) avec N le nombre de bit envoyé au total. Il n'a donc aucune information sur le secret grâce à l'anonymat garanti par le canal de diffusion.

<b> <u> Question 3 </u> </b>

Ces primitives permettent l'échange d'une clé cryptographique à distance sans l'intervention d'un tiers si on a confiance en la neutralité des primitives. En raison de la facilité avec laquelle des espions peuvent surveiller les canaux de communication, ce processus permet d'échanger un message en clair permettant de créer une clé privée commune aux deux utilisateurs, qui permettra ensuite de communiquer des messages codés.